# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thunder bay,48.4001,-89.3168,273.14,51,0,1.54,CA,1711939294
1,1,saipan,15.1355,145.7010,302.54,1,75,9.77,MP,1711939461
2,2,punta arenas,-53.1500,-70.9167,277.21,93,75,3.60,CL,1711939369
3,3,invercargill,-46.4000,168.3500,287.05,75,97,2.24,NZ,1711939392
4,4,port-aux-francais,-49.3500,70.2167,278.00,76,100,9.87,TF,1711939461


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
total_map_plot = city_data_df.hvplot.points('Lng',
                                            'Lat', 
                                            geo = True, 
                                            tiles = 'OSM',
                                            frame_width = 700,
                                            frame_height = 500,
                                            size = 'Humidity',
                                            color = 'City')

# Display the map
total_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp'] < 300.15) & (city_data_df['Max Temp'] > 294.15) & 
                            (city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 4.5),:]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,salalah,17.0151,54.0924,298.20,65,0,1.03,OM,1711939464
78,78,olinda,-8.0089,-34.8553,300.00,100,0,1.54,BR,1711939468
169,169,umluj,25.0213,37.2685,299.51,48,0,4.16,SA,1711939477
229,229,conde,-7.2597,-34.9075,297.85,94,0,0.00,BR,1711939482
307,307,al kharijah,25.4514,30.5464,294.55,28,0,3.33,EG,1711939489
310,310,chichen-itza,20.6667,-88.5667,297.51,71,0,3.02,MX,1711939489
320,320,atlantic beach,30.3344,-81.3987,294.85,76,0,3.60,US,1711939490
394,394,capayan,-28.7610,-66.0502,296.76,61,0,1.07,AR,1711939497
474,474,snyder,32.7179,-100.9176,297.34,33,0,3.60,US,1711939505


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.loc[:,["City","Country","Lat","Lng","Humidity"]]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,salalah,OM,17.0151,54.0924,65,
78,olinda,BR,-8.0089,-34.8553,100,
169,umluj,SA,25.0213,37.2685,48,
229,conde,BR,-7.2597,-34.9075,94,
307,al kharijah,EG,25.4514,30.5464,28,
310,chichen-itza,MX,20.6667,-88.5667,71,
320,atlantic beach,US,30.3344,-81.3987,76,
394,capayan,AR,-28.7610,-66.0502,61,
474,snyder,US,32.7179,-100.9176,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey":geoapify_key 
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index,'Lat']
    lon = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salalah - nearest hotel: Muscat International Hotel
olinda - nearest hotel: Fidji Motel
umluj - nearest hotel: No hotel found
conde - nearest hotel: Paladino
al kharijah - nearest hotel: Kharga Hotel
chichen-itza - nearest hotel: Hotel Villas Arqueológicas
atlantic beach - nearest hotel: One Ocean Resort & Spa
capayan - nearest hotel: Hosteria Concepción
snyder - nearest hotel: Days Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
34,salalah,OM,17.0151,54.0924,65,Muscat International Hotel
78,olinda,BR,-8.0089,-34.8553,100,Fidji Motel
169,umluj,SA,25.0213,37.2685,48,No hotel found
229,conde,BR,-7.2597,-34.9075,94,Paladino
307,al kharijah,EG,25.4514,30.5464,28,Kharga Hotel
310,chichen-itza,MX,20.6667,-88.5667,71,Hotel Villas Arqueológicas
320,atlantic beach,US,30.3344,-81.3987,76,One Ocean Resort & Spa
394,capayan,AR,-28.7610,-66.0502,61,Hosteria Concepción
474,snyder,US,32.7179,-100.9176,33,Days Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points('Lng',
                                        'Lat', 
                                        geo = True, 
                                        tiles = 'OSM',
                                        frame_width = 700,
                                        frame_height = 500,
                                        size = 'Humidity',
                                        color = 'City',
                                        hover_cols = ['Hotel Name','Country'])

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)